# Finetuning models

In [1]:
# Import necessary modules
import torch
from torch import nn, optim
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torchvision.datasets import FashionMNIST
from timeit import default_timer as timer
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
import os
import json

In [2]:
# Set random seeds for reproducibility
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

In [3]:
# Define device (use "cuda" if available, otherwise "cpu" or "mps")
device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
print(f"[INFO] Using device: {device}")

[INFO] Using device: mps


In [4]:
import sys
import os

# Add the src folder to the Python path
sys.path.insert(0, os.path.abspath("../src"))

## Setup and Initialization

In [5]:
# Define the loss function
loss_fn = nn.CrossEntropyLoss()

# Define the number of epochs for fine-tuning
NUM_EPOCHS = 10

# Set the number of classes for the classification task
NUM_CLASSES = 10

# Define the paths for the preprocessed CSV files
csv_dir = "../data_preparation"
train_csv_path = os.path.join(csv_dir, "train_data.csv")
validation_csv_path = os.path.join(csv_dir, "validation_data.csv")
test_csv_path = os.path.join(csv_dir, "test_data.csv")

# Function to load CSV data into a TensorDataset
def load_csv_to_dataset(csv_path):
    """
    Loads data from a CSV file into a PyTorch TensorDataset.

    Args:
        csv_path (str): Path to the CSV file.

    Returns:
        TensorDataset: A dataset containing features and labels as tensors.
    """
    df = pd.read_csv(csv_path)
    labels = torch.tensor(df.iloc[:, 0].values, dtype=torch.long)
    features = torch.tensor(df.iloc[:, 1:].values, dtype=torch.float32).reshape(-1, 1, 28, 28)
    return TensorDataset(features, labels)

# Load datasets
train_data = load_csv_to_dataset(train_csv_path)
validation_data = load_csv_to_dataset(validation_csv_path)
test_data = load_csv_to_dataset(test_csv_path)

print("\n✅ Data loading completed successfully!")
print(f"Training samples: {len(train_data)}")
print(f"Validation samples: {len(validation_data)}")
print(f"Test samples: {len(test_data)}")

# Quick data check
sample_img, sample_label = train_data[0]
print(f"Sample image shape: {sample_img.shape}, Sample label: {sample_label}")


✅ Data loading completed successfully!
Training samples: 48000
Validation samples: 12000
Test samples: 10000
Sample image shape: torch.Size([1, 28, 28]), Sample label: 1


## Hyperparameter Tuning Setup

In [6]:
# Hyperparameter Tuning Grid Setup

# Define possible batch sizes to explore
BATCH_SIZES = [32, 64]
# BATCH_SIZES = [32, 64, 128]

# Define learning rates to try for fine-tuning
LEARNING_RATES = [1e-5, 5e-6]
#LEARNING_RATES = [1e-5, 5e-6, 1e-6]

# Define patience values for early stopping
PATIENCE_VALUES = [2, 3]
#PATIENCE_VALUES = [2, 3, 5]

# Function to create DataLoaders with a specified batch size
def create_dataloaders(batch_size):
    """
    Creates DataLoaders for training, validation, and test datasets.

    Args:
        batch_size (int): Batch size for the DataLoaders.

    Returns:
        Tuple of DataLoaders (train_loader, val_loader, test_loader).
    """
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(validation_data, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)
    return train_loader, val_loader, test_loader

# Test DataLoader creation with the default batch size of 32
default_batch_size = 32
train_loader, val_loader, test_loader = create_dataloaders(batch_size=default_batch_size)

# Quick check for one batch of training data
train_features_batch, train_labels_batch = next(iter(train_loader))
print(f"Feature batch shape: {train_features_batch.shape}, Labels batch shape: {train_labels_batch.shape}")

# Initialize a dictionary to store results of all hyperparameter configurations
tuning_results = {}

print("\n✅ Hyperparameter tuning setup completed successfully!")

Feature batch shape: torch.Size([32, 1, 28, 28]), Labels batch shape: torch.Size([32])

✅ Hyperparameter tuning setup completed successfully!


## Loading Pre-Trained Models for Fine-Tuning

In [7]:
# Import necessary modules
import os
import torch
from torch import optim

# Define the directory where pre-trained models are saved
models_dir = "../models/all_models"

# Function to load a model and its weights
# Function to load a model and its weights
def load_model(model_class, model_name, in_channels=1, num_classes=NUM_CLASSES, hidden_units=20):
    """
    Loads a pre-trained model and its weights using the state dictionary.

    Args:
        model_class: The class of the model to be loaded (e.g., MiniCNN, TinyVGG, ResNet).
        model_name (str): Name of the model file (without extension).
        in_channels (int): Number of input channels (default is 1 for grayscale images).
        num_classes (int): Number of output classes.
        hidden_units (int): Number of hidden units (used for TinyVGG).

    Returns:
        model: The loaded model instance with weights.
    """
    model_path = os.path.join(models_dir, f"{model_name}_weights.pth")
    
    # Initialize the model based on its class
    if model_class == MiniCNN:
        model = model_class(in_channels=in_channels, num_classes=num_classes)
    elif model_class == TinyVGG:
        model = model_class(in_channels=in_channels, hidden_units=hidden_units, num_classes=num_classes)
    elif model_class == ResNet:
        model = model_class(BasicBlock, [2, 2, 2, 2], num_classes=num_classes)
    else:
        raise ValueError(f"Unknown model class: {model_class}")

    # Load the state dictionary
    model.load_state_dict(torch.load(model_path, weights_only=True))
    model.to(device)
    model.eval()  # Set the model to evaluation mode
    print(f"✅ {model_name} loaded successfully from: {model_path}")
    return model


# Load the Mini CNN model
from model_definitions import MiniCNN
mini_cnn_model = load_model(MiniCNN, "mini_cnn_model")

# Load the Tiny VGG model
from model_definitions import TinyVGG
tiny_vgg_model = load_model(TinyVGG, "tiny_vgg_model")

# Load the ResNet model
from model_definitions import ResNet, BasicBlock
resnet_model = load_model(ResNet, "resnet_model")

# Define a function to create an optimizer with a smaller learning rate for fine-tuning
def setup_optimizer(model, learning_rate):
    """
    Sets up an Adam optimizer for a given model with a specified learning rate.

    Args:
        model: The model to optimize.
        learning_rate (float): The learning rate for the optimizer.

    Returns:
        optimizer: A PyTorch Adam optimizer instance.
    """
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    print(f"🔧 Optimizer set up with learning rate: {learning_rate}")
    return optimizer

# Setup optimizers for each model with a smaller learning rate for fine-tuning
mini_cnn_optimizer = setup_optimizer(mini_cnn_model, learning_rate=1e-5)
tiny_vgg_optimizer = setup_optimizer(tiny_vgg_model, learning_rate=1e-5)
resnet_optimizer = setup_optimizer(resnet_model, learning_rate=1e-6)

# Quick summary of models and optimizers
print("\n[INFO] Pre-trained models and optimizers are ready for fine-tuning.")
print(f"Mini CNN model: {mini_cnn_model}")
print(f"Tiny VGG model: {tiny_vgg_model}")
print(f"ResNet model: {resnet_model}")


✅ mini_cnn_model loaded successfully from: ../models/all_models/mini_cnn_model_weights.pth
✅ tiny_vgg_model loaded successfully from: ../models/all_models/tiny_vgg_model_weights.pth
✅ resnet_model loaded successfully from: ../models/all_models/resnet_model_weights.pth
🔧 Optimizer set up with learning rate: 1e-05
🔧 Optimizer set up with learning rate: 1e-05
🔧 Optimizer set up with learning rate: 1e-06

[INFO] Pre-trained models and optimizers are ready for fine-tuning.
Mini CNN model: MiniCNN(
  (conv_block): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=1568, out_features=64, 

### Fine-Tuning Function

In [8]:
from typing import Dict, List
from tqdm.auto import tqdm

def fine_tune_model(
    model: torch.nn.Module,
    train_dataloader: torch.utils.data.DataLoader,
    val_dataloader: torch.utils.data.DataLoader,
    test_dataloader: torch.utils.data.DataLoader,
    optimizer: torch.optim.Optimizer,
    loss_fn: torch.nn.Module,
    epochs: int,
    device: torch.device,
    patience: int = 3,
    min_delta: float = 0.001,
    unfreeze_layers: bool = False
) -> Dict[str, List[float]]:
    """
    Fine-tunes a PyTorch model with optional unfreezing of layers.

    Args:
        model: The model to be fine-tuned.
        train_dataloader: DataLoader for the training data.
        val_dataloader: DataLoader for the validation data.
        test_dataloader: DataLoader for the test data.
        optimizer: Optimizer for minimizing the loss.
        loss_fn: Loss function to use.
        epochs: Maximum number of epochs for fine-tuning.
        device: Device for computations (e.g., "cuda" or "cpu").
        patience: Number of epochs to wait for improvement before early stopping.
        min_delta: Minimum change in validation loss to qualify as an improvement.
        unfreeze_layers: Whether to unfreeze certain layers of the model.

    Returns:
        A dictionary containing training, validation, and testing metrics.
    """
    # Unfreeze certain layers if specified
    if unfreeze_layers:
        print("\n🔓 Unfreezing certain layers for fine-tuning...")
        if isinstance(model, ResNet):
            for param in model.layer4.parameters():
                param.requires_grad = True
        elif isinstance(model, TinyVGG):
            for param in model.conv_block_2.parameters():
                param.requires_grad = True
        print("✅ Layers unfrozen successfully.")

    # Initialize results dictionary
    results = {
        "train_loss": [], "train_acc": [],
        "val_loss": [], "val_acc": [],
        "test_loss": [], "test_acc": []
    }

    # Initialize early stopping variables
    best_val_loss = float('inf')
    counter = 0
    best_model_weights = model.state_dict()

    # Loop through epochs
    for epoch in tqdm(range(epochs)):
        # Training step
        train_loss, train_acc = train_step(model, train_dataloader, loss_fn, optimizer, device)

        # Validation step
        val_loss, val_acc = validation_step(model, val_dataloader, loss_fn, device)

        # Testing step
        test_loss, test_acc = test_step(model, test_dataloader, loss_fn, device)

        # Print epoch metrics
        print(
            f"Epoch: {epoch + 1} | "
            f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | "
            f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f} | "
            f"Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.4f}"
        )

        # Store metrics
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["val_loss"].append(val_loss)
        results["val_acc"].append(val_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

        # Early stopping logic
        if val_loss < best_val_loss - min_delta:
            best_val_loss = val_loss
            best_model_weights = model.state_dict()
            counter = 0
            print("✅ Improvement in validation loss. Best model weights updated.")
        else:
            counter += 1
            print(f"No improvement in validation loss. Early stopping counter: {counter}/{patience}")

        # Check if early stopping should be triggered
        if counter >= patience:
            print(f"⏹️ Early stopping triggered at epoch {epoch + 1}.")
            break

    # Load the best model weights
    model.load_state_dict(best_model_weights)

    return results


In [9]:
from itertools import product
from utils import train_step, validation_step, test_step
# Helper function to create and load a model
def get_model_and_optimizer(model_class, model_name, learning_rate, hidden_units=20):
    """
    Initializes the model, loads pre-trained weights, and sets up the optimizer.
    """
    model = load_model(model_class, model_name)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    print(f"🔧 Loaded {model_name} and set up optimizer with learning rate: {learning_rate}")
    return model, optimizer

# Helper function to fine-tune a given model
def fine_tune_and_evaluate(model, optimizer, train_loader, val_loader, test_loader, patience):
    """
    Fine-tunes the model and returns the fine-tuning results.
    """
    return fine_tune_model(
        model=model,
        train_dataloader=train_loader,
        val_dataloader=val_loader,
        test_dataloader=test_loader,
        optimizer=optimizer,
        loss_fn=loss_fn,
        epochs=NUM_EPOCHS,
        device=device,
        patience=patience,
        min_delta=0.001,
        unfreeze_layers=True
    )

# Initialize a dictionary to store the best results
best_results = {
    "model_name": None,
    "best_val_acc": 0.0,
    "best_params": None,
    "best_model": None
}

# Directory to save fine-tuning results
results_dir = "../results/fine_tuning_results"
os.makedirs(results_dir, exist_ok=True)  # Ensure the directory exists

# Iterate over all combinations of hyperparameters
for batch_size, learning_rate, patience in product(BATCH_SIZES, LEARNING_RATES, PATIENCE_VALUES):
    print(f"\n🔄 Tuning with Batch Size: {batch_size}, Learning Rate: {learning_rate}, Patience: {patience}")

    # Create DataLoaders
    train_loader, val_loader, test_loader = create_dataloaders(batch_size)

    # List of models to fine-tune
    model_configs = [
        (MiniCNN, "mini_cnn_model", learning_rate),
        (TinyVGG, "tiny_vgg_model", learning_rate),
        (ResNet, "resnet_model", learning_rate)
    ]

    # Loop through each model configuration
    for model_class, model_name, lr in model_configs:
        print(f"\n🔄 Fine-tuning {model_name}...")

        # Get model and optimizer
        model, optimizer = get_model_and_optimizer(model_class, model_name, lr)

        # Fine-tune the model
        results = fine_tune_and_evaluate(model, optimizer, train_loader, val_loader, test_loader, patience)

        # Save results to JSON file
        json_filename = f"{model_name}_bs{batch_size}_lr{learning_rate}_pat{patience}.json"
        json_path = os.path.join(results_dir, json_filename)
        
        with open(json_path, "w") as json_file:
            json.dump(results, json_file, indent=4)

        print(f"✅ Results saved for {model_name} at {json_path}")

        # Check if this is the best result
        if results["val_acc"][-1] > best_results["best_val_acc"]:
            best_results["model_name"] = model_name
            best_results["best_val_acc"] = results["val_acc"][-1]
            best_results["best_params"] = (batch_size, learning_rate, patience)
            best_results["best_model"] = model.state_dict()

# Summary of Best Results
print("\n🎉 Hyperparameter Tuning Completed!")
print(f"Best Model: {best_results['model_name']}")
print(f"Best Validation Accuracy: {best_results['best_val_acc']:.4f}")
print(f"Best Hyperparameters (Batch Size, Learning Rate, Patience): {best_results['best_params']}")

# Save the best model weights
best_model_path = "../models/best_model_weights/best_model_weights.pth"
torch.save(best_results["best_model"], best_model_path)
print(f"✅ Best model weights saved to: {best_model_path}")



🔄 Tuning with Batch Size: 32, Learning Rate: 1e-05, Patience: 2

🔄 Fine-tuning mini_cnn_model...
✅ mini_cnn_model loaded successfully from: ../models/all_models/mini_cnn_model_weights.pth
🔧 Loaded mini_cnn_model and set up optimizer with learning rate: 1e-05

🔓 Unfreezing certain layers for fine-tuning...
✅ Layers unfrozen successfully.


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | Train Loss: 0.1478 | Train Acc: 0.9464 | Val Loss: 0.2229 | Val Acc: 0.9196 | Test Loss: 0.2513 | Test Acc: 0.9113
✅ Improvement in validation loss. Best model weights updated.
Epoch: 2 | Train Loss: 0.1408 | Train Acc: 0.9492 | Val Loss: 0.2213 | Val Acc: 0.9216 | Test Loss: 0.2503 | Test Acc: 0.9124
✅ Improvement in validation loss. Best model weights updated.
Epoch: 3 | Train Loss: 0.1385 | Train Acc: 0.9500 | Val Loss: 0.2208 | Val Acc: 0.9217 | Test Loss: 0.2503 | Test Acc: 0.9130
No improvement in validation loss. Early stopping counter: 1/2
Epoch: 4 | Train Loss: 0.1369 | Train Acc: 0.9501 | Val Loss: 0.2205 | Val Acc: 0.9215 | Test Loss: 0.2500 | Test Acc: 0.9131
No improvement in validation loss. Early stopping counter: 2/2
⏹️ Early stopping triggered at epoch 4.
✅ Results saved for mini_cnn_model at ../results/fine_tuning_results/mini_cnn_model_bs32_lr1e-05_pat2.json

🔄 Fine-tuning tiny_vgg_model...
✅ tiny_vgg_model loaded successfully from: ../models/all_models/ti

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | Train Loss: 0.1731 | Train Acc: 0.9375 | Val Loss: 0.2264 | Val Acc: 0.9197 | Test Loss: 0.2516 | Test Acc: 0.9096
✅ Improvement in validation loss. Best model weights updated.
Epoch: 2 | Train Loss: 0.1645 | Train Acc: 0.9416 | Val Loss: 0.2240 | Val Acc: 0.9207 | Test Loss: 0.2496 | Test Acc: 0.9115
✅ Improvement in validation loss. Best model weights updated.
Epoch: 3 | Train Loss: 0.1617 | Train Acc: 0.9425 | Val Loss: 0.2230 | Val Acc: 0.9207 | Test Loss: 0.2486 | Test Acc: 0.9115
No improvement in validation loss. Early stopping counter: 1/2
Epoch: 4 | Train Loss: 0.1600 | Train Acc: 0.9431 | Val Loss: 0.2223 | Val Acc: 0.9207 | Test Loss: 0.2483 | Test Acc: 0.9113
✅ Improvement in validation loss. Best model weights updated.
Epoch: 5 | Train Loss: 0.1586 | Train Acc: 0.9437 | Val Loss: 0.2217 | Val Acc: 0.9210 | Test Loss: 0.2477 | Test Acc: 0.9118
No improvement in validation loss. Early stopping counter: 1/2
Epoch: 6 | Train Loss: 0.1575 | Train Acc: 0.9441 | Val Lo

...

Epoch: 1 | Train Loss: 0.1541 | Train Acc: 0.9435 | Val Loss: 0.2261 | Val Acc: 0.9185 | Test Loss: 0.2538 | Test Acc: 0.9110
✅ Improvement in validation loss. Best model weights updated.
Epoch: 2 | Train Loss: 0.1449 | Train Acc: 0.9475 | Val Loss: 0.2235 | Val Acc: 0.9197 | Test Loss: 0.2517 | Test Acc: 0.9114
✅ Improvement in validation loss. Best model weights updated.
Epoch: 3 | Train Loss: 0.1427 | Train Acc: 0.9489 | Val Loss: 0.2225 | Val Acc: 0.9206 | Test Loss: 0.2512 | Test Acc: 0.9124
✅ Improvement in validation loss. Best model weights updated.
Epoch: 4 | Train Loss: 0.1413 | Train Acc: 0.9491 | Val Loss: 0.2218 | Val Acc: 0.9215 | Test Loss: 0.2507 | Test Acc: 0.9122
No improvement in validation loss. Early stopping counter: 1/3
Epoch: 5 | Train Loss: 0.1402 | Train Acc: 0.9493 | Val Loss: 0.2214 | Val Acc: 0.9215 | Test Loss: 0.2505 | Test Acc: 0.9126
✅ Improvement in validation loss. Best model weights updated.
Epoch: 6 | Train Loss: 0.1393 | Train Acc: 0.9496 | Val Los

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | Train Loss: 0.1802 | Train Acc: 0.9345 | Val Loss: 0.2336 | Val Acc: 0.9166 | Test Loss: 0.2589 | Test Acc: 0.9071
✅ Improvement in validation loss. Best model weights updated.
Epoch: 2 | Train Loss: 0.1704 | Train Acc: 0.9385 | Val Loss: 0.2284 | Val Acc: 0.9186 | Test Loss: 0.2532 | Test Acc: 0.9091
✅ Improvement in validation loss. Best model weights updated.
Epoch: 3 | Train Loss: 0.1668 | Train Acc: 0.9405 | Val Loss: 0.2263 | Val Acc: 0.9201 | Test Loss: 0.2510 | Test Acc: 0.9104
✅ Improvement in validation loss. Best model weights updated.
Epoch: 4 | Train Loss: 0.1650 | Train Acc: 0.9414 | Val Loss: 0.2252 | Val Acc: 0.9200 | Test Loss: 0.2499 | Test Acc: 0.9107
✅ Improvement in validation loss. Best model weights updated.
Epoch: 5 | Train Loss: 0.1636 | Train Acc: 0.9415 | Val Loss: 0.2245 | Val Acc: 0.9206 | Test Loss: 0.2494 | Test Acc: 0.9105
No improvement in validation loss. Early stopping counter: 1/3
Epoch: 6 | Train Loss: 0.1626 | Train Acc: 0.9420 | Val Los

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | Train Loss: 0.1011 | Train Acc: 0.9636 | Val Loss: 0.1890 | Val Acc: 0.9326 | Test Loss: 0.2149 | Test Acc: 0.9239
✅ Improvement in validation loss. Best model weights updated.
Epoch: 2 | Train Loss: 0.0869 | Train Acc: 0.9688 | Val Loss: 0.1835 | Val Acc: 0.9352 | Test Loss: 0.2090 | Test Acc: 0.9260
✅ Improvement in validation loss. Best model weights updated.
Epoch: 3 | Train Loss: 0.0805 | Train Acc: 0.9710 | Val Loss: 0.1820 | Val Acc: 0.9360 | Test Loss: 0.2075 | Test Acc: 0.9268
✅ Improvement in validation loss. Best model weights updated.
Epoch: 4 | Train Loss: 0.0771 | Train Acc: 0.9725 | Val Loss: 0.1812 | Val Acc: 0.9369 | Test Loss: 0.2068 | Test Acc: 0.9278
No improvement in validation loss. Early stopping counter: 1/3
Epoch: 5 | Train Loss: 0.0736 | Train Acc: 0.9740 | Val Loss: 0.1807 | Val Acc: 0.9374 | Test Loss: 0.2057 | Test Acc: 0.9280
✅ Improvement in validation loss. Best model weights updated.
Epoch: 6 | Train Loss: 0.0708 | Train Acc: 0.9749 | Val Los